# Installing Libraries for Fine-turning



NOTE: Before executing cells **change runtime type > T4 GPU** or any powerful GPU available to you. Because training the model is a very expensive and hardware resources consuming task.

In [ ]:
# install libraries
#SentencePiece is an unsupervised text tokenizer and detokenizer mainly for Neural Network-based,
#text generation systems where the vocabulary size is predetermined prior to the neural model training.
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs
!pip install sacrebleu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
Reading package lists... Done
Building dependency tree... Done
Reading state inform

In [ ]:
# you must create huggingface account for purpose of token validation and login

from huggingface_hub import notebook_login
notebook_login()

# LOAD THE DATA

In [ ]:

from datasets import load_dataset
raw_datasets = load_dataset("GODCOMPLEX/nagamese2english")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

final.csv:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7950 [00:00<?, ? examples/s]

In [ ]:
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['English', 'Nagamese'],
        num_rows: 7950
    })
})

In [ ]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.8, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['English', 'Nagamese'],
        num_rows: 6360
    })
    test: Dataset({
        features: ['English', 'Nagamese'],
        num_rows: 1590
    })
})

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
split_datasets["train"][1]

{'English': 'Jesus answered and said to them, "Even if I testify about myself, my testimony is true. I know where I came from and where I am going, but you do not know where I came from or where I am going.',
 'Nagamese': 'Jisu taikhan ke jowab di koise, “Jodi Moi nijor karone sakhi dile bi, Moi laga sakhi hosa ase. Kilemane Moi kot pora ahise aru kot te jabole ase itu Moi jane, kintu tumi khan to Moi kot pora ahise aru kot te jabole ase najane.'}

# DATA PROCESSING

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "facebook/mbart-large-50"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="en_XX", tgt_lang="ng_XX")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
en_sentence = split_datasets["train"][1]["English"]
ng_sentence = split_datasets["train"][1]["Nagamese"]

inputs = tokenizer(en_sentence, text_target=ng_sentence)
inputs

{'input_ids': [250004, 9716, 194663, 136, 2804, 47, 2856, 4, 44, 647, 1353, 2174, 87, 3034, 40383, 1672, 35978, 4, 759, 3034, 2414, 299, 83, 29568, 5, 87, 3714, 7440, 87, 21449, 1295, 136, 7440, 87, 444, 7730, 4, 1284, 398, 54, 959, 3714, 7440, 87, 21449, 1295, 707, 7440, 87, 444, 7730, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [3, 33443, 34, 1013, 124880, 311, 741, 72947, 45, 298, 3075, 4, 52, 16604, 428, 32129, 300, 15503, 12208, 86, 57, 52568, 26052, 333, 4, 32129, 6327, 57, 52568, 3080, 11, 10, 184, 5, 1519, 41092, 86, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 752, 32129, 47363, 4, 200, 18461, 111788, 70144, 47, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 2034, 6130, 5, 2]}

In [ ]:
max_length = 128

def preprocess_function(examples):
    inputs = [ex for ex in examples["English"]]
    targets = [ex for ex in examples["Nagamese"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )

    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/6360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model_checkpoint = "facebook/mbart-large-50"
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch["labels"]

tensor([[     3,  33443,     34,   1013, 124880,    311,    741,  72947,     45,
            298,   3075,      4,     52,  16604,    428,  32129,    300,  15503,
          12208,     86,     57,  52568,  26052,    333,      4,  32129,   6327,
             57,  52568,   3080,     11,     10,    184,      5,   1519,  41092,
             86,  32129,   1975,    196,     11,   1263,   3075,  72205,   1975,
            120,     79,  61217,     10,    184,    752,  32129,  47363,      4,
            200,  18461, 111788,  70144,     47,  32129,   1975,    196,     11,
           1263,   3075,  72205,   1975,    120,     79,  61217,     10,    184,
           2034,   6130,      5,      2],
        [     3,  10208,  72205,   1013,   6327,     57,   4450,  70144, 185709,
             53,   7366,  72205,  30733,   4526,   6327,  86864,  40681,    133,
            764,     79,   3075,      4,    200,  41092,     86,    663,   3137,
            516,   1913,    192,    196,     11,   1013, 124880,   

In [ ]:
batch["decoder_input_ids"]

tensor([[     2,      3,  33443,     34,   1013, 124880,    311,    741,  72947,
             45,    298,   3075,      4,     52,  16604,    428,  32129,    300,
          15503,  12208,     86,     57,  52568,  26052,    333,      4,  32129,
           6327,     57,  52568,   3080,     11,     10,    184,      5,   1519,
          41092,     86,  32129,   1975,    196,     11,   1263,   3075,  72205,
           1975,    120,     79,  61217,     10,    184,    752,  32129,  47363,
              4,    200,  18461, 111788,  70144,     47,  32129,   1975,    196,
             11,   1263,   3075,  72205,   1975,    120,     79,  61217,     10,
            184,   2034,   6130,      5],
        [     2,      3,  10208,  72205,   1013,   6327,     57,   4450,  70144,
         185709,     53,   7366,  72205,  30733,   4526,   6327,  86864,  40681,
            133,    764,     79,   3075,      4,    200,  41092,     86,    663,
           3137,    516,   1913,    192,    196,     11,   1013, 12

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[3, 33443, 34, 1013, 124880, 311, 741, 72947, 45, 298, 3075, 4, 52, 16604, 428, 32129, 300, 15503, 12208, 86, 57, 52568, 26052, 333, 4, 32129, 6327, 57, 52568, 3080, 11, 10, 184, 5, 1519, 41092, 86, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 752, 32129, 47363, 4, 200, 18461, 111788, 70144, 47, 32129, 1975, 196, 11, 1263, 3075, 72205, 1975, 120, 79, 61217, 10, 184, 2034, 6130, 5, 2]
[3, 10208, 72205, 1013, 6327, 57, 4450, 70144, 185709, 53, 7366, 72205, 30733, 4526, 6327, 86864, 40681, 133, 764, 79, 3075, 4, 200, 41092, 86, 663, 3137, 516, 1913, 192, 196, 11, 1013, 124880, 311, 14237, 120, 502, 9254, 74473, 61217, 3513, 107760, 13, 5, 2]


In [ ]:
import evaluate
import numpy as np

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

# FINE-Tuning The Model

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"mbart-50-finetuned-eng-to-naga",  # mBART-50 is a multilingual Sequence-to-Sequence model. It was introduced to show that multilingual translation models can be created through multilingual fine-tuning.
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# A wandb.ai profile is alos needed to create for API Key....

trainer.evaluate(max_length=max_length)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 39


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 10.652088165283203,
 'eval_model_preparation_time': 0.0078,
 'eval_bleu': 0.20485897823256105,
 'eval_runtime': 1487.8241,
 'eval_samples_per_second': 1.069,
 'eval_steps_per_second': 1.069}

In [ ]:
trainer.train()

Step,Training Loss
500,4.327400
1000,2.828800
1500,5.228100
2000,2.541600
2500,2.337500
3000,2.293100
3500,2.052400
4000,1.909200
4500,1.896100
5000,1.817300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


In [ ]:
trainer.evaluate(max_length=max_length)

In [ ]:

trainer.push_to_hub(tags="translation", commit_message="Training complete")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GODCOMPLEX/mbart-50-finetuned-eng-to-naga/commit/de92b2aa47650746dba5750a142c5badf69891ba', commit_message='Training complete', commit_description='', oid='de92b2aa47650746dba5750a142c5badf69891ba', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:

from transformers import pipeline

model_checkpoint = "GODCOMPLEX/mbart-50-finetuned-eng-to-naga"
translator = pipeline("translation", model=model_checkpoint, src_lang="en_XX", tgt_lang="ng_XX")

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/226 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:

translator("The boat was in the sea")

[{'translation_text': 'Aru naw to samundar te thakise'}]

In [ ]:

translator(
    "When he was walking beside the Sea of Galilee, he saw Simon and Andrew the brother of Simon casting a net in the sea, for they were fishermen."
)


[{'translation_text': 'Jitia Tai Galilee laga samundar kinar te berai thakise, Tai Simon aru tai laga bhai Andrew ke samundar te jal phelai thaka dikhise, kilemane tai khan maas dhora manu asele.'}]

In [ ]:
translator(" eating fish")

[{'translation_text': 'maas kha luwa kora'}]

In [ ]:

split_datasets['validation'][1]

{'English': 'For by the grace that was given to me I say to everyone among you: Do not think of yourself more highly than you ought, but rather, think with sober judgment, each according to the measure of faith that God has given you.',
 'Nagamese': 'Moi koi ase, kilemane moi ke anugraha dise, itu pora sob logote koi ase, apnikhan majot te kun manu pora bi nijor ki ase itu pora dangor bhabona nakoribi, kintu nomro hoikena nijor ke Isor pora kiman biswas dise, itu hisab te bhabona kori thakibi.'}

In [ ]:
translator(split_datasets["validation"][7]["English"])

[{'translation_text': 'Jitia Jisu taikhan laga biswas dikhise, Tai koise, “Manu, tumi laga paap khan sob maph kori dise.”'}]